In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from scipy.stats import chi2_contingency, chi2, mannwhitneyu, shapiro, kruskal
import statsmodels.api as sa 
import scikit_posthocs as sp

## 1.Загружаем данные и делаем предподготовку

In [2]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/406362/churn.csv')

In [3]:
df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4


In [5]:
df.isnull().sum()

avg_dist                     0
avg_rating_by_driver       201
avg_rating_of_driver      8122
avg_surge                    0
city                         0
last_trip_date               0
phone                      396
signup_date                  0
surge_pct                    0
trips_in_first_30_days       0
luxury_car_user              0
weekday_pct                  0
dtype: int64

In [6]:
df.dtypes

avg_dist                  float64
avg_rating_by_driver      float64
avg_rating_of_driver      float64
avg_surge                 float64
city                       object
last_trip_date             object
phone                      object
signup_date                object
surge_pct                 float64
trips_in_first_30_days      int64
luxury_car_user              bool
weekday_pct               float64
dtype: object

In [7]:
# Изменяем тип данных для колонок с датами
df['last_trip_date'] = pd.to_datetime(df.last_trip_date)
df['signup_date'] = pd.to_datetime(df.signup_date)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   avg_dist                50000 non-null  float64       
 1   avg_rating_by_driver    49799 non-null  float64       
 2   avg_rating_of_driver    41878 non-null  float64       
 3   avg_surge               50000 non-null  float64       
 4   city                    50000 non-null  object        
 5   last_trip_date          50000 non-null  datetime64[ns]
 6   phone                   49604 non-null  object        
 7   signup_date             50000 non-null  datetime64[ns]
 8   surge_pct               50000 non-null  float64       
 9   trips_in_first_30_days  50000 non-null  int64         
 10  luxury_car_user         50000 non-null  bool          
 11  weekday_pct             50000 non-null  float64       
dtypes: bool(1), datetime64[ns](2), float64(6), int

In [9]:
df.describe(include='object')

,city,phone
count,50000,49604
unique,3,2
top,Winterfell,iPhone
freq,23336,34582


In [10]:
df.describe(include='datetime')

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,last_trip_date,signup_date
count,50000,50000
unique,182,31
top,2014-06-29 00:00:00,2014-01-18 00:00:00
freq,2036,2948
first,2014-01-01 00:00:00,2014-01-01 00:00:00
last,2014-07-01 00:00:00,2014-01-31 00:00:00


---

## 2. Введем переменную churn

Создаем лейбл **churn** – пользователь ушел, если не был активен последние 30 дней

In [11]:
df.last_trip_date.max()

Timestamp('2014-07-01 00:00:00')

In [12]:
df['days_since_last_trip'] = df.last_trip_date.max() - df.last_trip_date

In [13]:
df['days_since_last_trip'] = df['days_since_last_trip'].dt.days

In [31]:
df['churn'] = df.days_since_last_trip.apply(lambda x: 'churn' if x > 30 else 'not_churn')
df[['days_since_last_trip', 'churn']].head()

,days_since_last_trip,churn
0,14,not_churn
1,57,churn
2,175,churn
3,2,not_churn
4,108,churn


In [21]:
df.churn.value_counts(normalize=True).mul(100)

churn        62.392
not_churn    37.608
Name: churn, dtype: float64

In [22]:
fig = px.histogram(df, x='churn', histnorm='probability density')
fig.show()

Видим, что очень много пользователей не использовали сервис в последнем месяце. Нужно разобраться, какие факторы могут влиять на отток водителей

---

## 3. Проверим влияние города на churn

In [23]:
fig = px.histogram(df[['churn', 'city']].dropna(), x='churn', 
                   color='city')
fig.show()

Делать вывод только по графику – не очень хорошо, поэтому проверим нашу гипотезу с помощью статистического теста.

Есть две категориальные переменные → нужен хи-квадрат

- $H_0$: взаимосвязи между переменными нет 
- $H_1$: взаимосвязь есть

In [24]:
pd.crosstab(df.churn, df.city)

city,Astapor,King's Landing,Winterfell
churn,,,
churn,12306,3767,15123
not_churn,4228,6363,8213


In [25]:
stat, p, dof, expected = chi2_contingency(pd.crosstab(df.churn, df.city))

In [26]:
stat, p, dof, expected

(3821.5510225559633,
 0.0,
 2,
 array([[10315.89328,  6320.3096 , 14559.79712],
        [ 6218.10672,  3809.6904 ,  8776.20288]]))

In [27]:
prob = 0.95
alpha = 1.0 - prob
if p <= alpha:
    print('Отклоняем H0')
else:
    print('Не отклоняем H0')

Отклоняем H0


Нулевая гипотеза **отклоняется**, поскольку p-value < 0.05. Значит, **взаимосвязь между churn и city есть**.

---

## 4. Проверим разницу в активности в первые 30 дней с момента регистрации между водителями из разных городов

In [28]:
# Уберем символ (') из названий городов в строке для дальнейшего удобства работы с ними 
df['city'] = df.city.str.replace("'", '')

In [30]:
df[['city', 'trips_in_first_30_days']].head()

,city,trips_in_first_30_days
0,Kings Landing,4
1,Astapor,0
2,Astapor,3
3,Kings Landing,9
4,Winterfell,14


Сформируем выборки по городам, и проверим распределение в них на нормальность

In [34]:
ast_sample = df.query('city == "Astapor"').trips_in_first_30_days.sample(1000, random_state=17)
stat, p = shapiro(ast_sample)
print(stat, p)

0.5414707660675049 4.203895392974451e-45


In [35]:
kngsl_sample = df.query('city == "Kings Landing"').trips_in_first_30_days.sample(1000, random_state=17)
stat, p = shapiro(kngsl_sample)
print(stat, p)

0.6132159233093262 1.6829594556541053e-42


In [36]:
wntfl_sample = df.query('city == "Winterfell"').trips_in_first_30_days.sample(1000, random_state=17)
stat, p = shapiro(wntfl_sample)
print(stat, p)

0.5455795526504517 7.006492321624085e-45


Ни одно из распределений не является нормальным, поэтому для дисперсионного анализа будем использовать непараметрический аналог ANOVA – критерий Краскела-Уоллиса.

In [37]:
ast = df.query('city == "Astapor"').trips_in_first_30_days
kngsl = df.query('city == "Kings Landing"').trips_in_first_30_days
wntfl = df.query('city == "Winterfell"').trips_in_first_30_days

In [38]:
kruskal(ast, kngsl, wntfl)

KruskalResult(statistic=221.32105325317454, pvalue=8.724567791938856e-49)

**Статистически значимые различия обнаружены**.

---

## 5.Проверим связь оттока с активностью в первые 30 дней после регистрации

In [39]:
df[['churn', 'trips_in_first_30_days']].head()

,churn,trips_in_first_30_days
0,not_churn,4
1,churn,0
2,churn,3
3,not_churn,9
4,churn,14


In [40]:
chrn_sample = df.query('churn == "churn"').trips_in_first_30_days.sample(1000, random_state=17)
nchrn_sample = df.query('churn == "not_churn"').trips_in_first_30_days.sample(1000, random_state=17)
chrn = df.query('churn == "churn"').trips_in_first_30_days
nchrn = df.query('churn == "not_churn"').trips_in_first_30_days

In [41]:
stat, p = shapiro(chrn_sample)
print(stat,p)

0.4566316604614258 0.0


In [42]:
stat, p = shapiro(nchrn_sample)
print(stat,p)

0.6462708711624146 3.391983062744652e-41


Распределение переменной **trips_in_first_30_days** не является нормальным, для проверки гипотезы буду использовать критерий Манна-Уитни, где нулевая гипотеза говорит о равенстве распределений.

In [48]:
np.mean(chrn)

2.6584818566482884

In [49]:
np.mean(nchrn)

4.3063178047224

In [45]:
mannwhitneyu(nchrn, chrn, alternative='greater')

MannwhitneyuResult(statistic=351842132.0, pvalue=0.0)

#### P-value ~ 0 в тесте Манна-Уитни, позволяет принять альтернативную о том, что распределения значимо различаются. Это значит, что отток связан с активностью в первые 30 дней после регистрации.